<a href="https://colab.research.google.com/github/josiah-mbao/Digit-Recognition-System/blob/main/cats_and_dogs_fin_tun.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import os

_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=_URL, extract=True)
PATH = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')

train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')


In [ ]:
from keras.applications import VGG16
from keras.models import Model
from keras.layers import Flatten, Dense

# Load the pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Freeze the convolutional layers to prevent their weights from being updated during training
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification layers on top of the base model
x = Flatten()(base_model.output)
x = Dense(512, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Display model summary
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# Define image preprocessing and augmentation options
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

validation_datagen = ImageDataGenerator(rescale=1./255)

# Define batch size
batch_size = 32

# Load and preprocess training data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='binary'
)

# Load and preprocess validation data
validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='binary'
)

# Example model training
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=5,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)


Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Epoch 1/5
62/62 [==============================] - 771s 12s/step - loss: 0.8294 - accuracy: 0.6692 - val_loss: 0.3447 - val_accuracy: 0.8468
Epoch 2/5
62/62 [==============================] - 758s 12s/step - loss: 0.4088 - accuracy: 0.8181 - val_loss: 0.3300 - val_accuracy: 0.8468
Epoch 3/5
62/62 [==============================] - 771s 12s/step - loss: 0.3542 - accuracy: 0.8425 - val_loss: 0.2854 - val_accuracy: 0.8760
Epoch 4/5
62/62 [==============================] - 771s 13s/step - loss: 0.3406 - accuracy: 0.8506 - val_loss: 0.2994 - val_accuracy: 0.8821
Epoch 5/5
62/62 [==============================] - 757s 12s/step - loss: 0.3216 - accuracy: 0.8608 - val_loss: 0.2919 - val_accuracy: 0.8790


# Model Testing with new datapoints

In [ ]:
#from keras.preprocessing import image
import numpy as np

# Load the image from your computer
image_path = '/content/1.jpg'
img = image.load_img(image_path, target_size=(150, 150))

img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.  # Normalize pixel values to [0, 1]

# Make predictions
predictions = model.predict(img_array)

# predicted class
if predictions[0] > 0.5:
    class_label = 'dog'
else:
    class_label = 'cat'

plt.imshow(img)
plt.title(f'Predicted: {class_label}')
plt.axis('off')
plt.show()

NameError: name 'model' is not defined